In [1]:
import skimage.measure as msr
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
from PIL import Image, ImageCms

import glob
import os
from pathlib import Path
import imageio
import cv2

In [ ]:
# For importing wmark.py from parent directory
# import sys
# sys.path.append("..")

In [2]:
# Importing class from wmark.py
from wmark import WaterMark

In [3]:
# Source Directory
srcFolder = 'TestSet/'
# Source Path
srcPth = Path(srcFolder).resolve()

# Destination Directory
# dstFolder = Path(srcFolder+'/marked/').resolve()
# Creating folder for marked images
# Path(dstFolder).mkdir(exist_ok=True)
# Destination Path
# dstPath = Path(dstFolder).resolve()

In [9]:
# Read original image
imgOriginal = WaterMark.imread('TestSet/1.tif')
    
# Mark image
wObject5 = WaterMark(5)
imgMarked = wObject5.embedMark(imgOriginal, factor = ImpactFactor)

# Creating variable for range of possible seeds 
seedRange = range(1, 10)

# Creating zeros Numpy Array and setting counter for results
results = np.zeros([len(seedRange), 1])
seeds = np.zeros([len(seedRange), 1])
counter = 0

for i in seedRange:
    
    wObject = WaterMark(i)

    metricMarked = wObject.decodeMark(imgMarked, 'CORR')
    results[counter, 0] = metricMarked
        
    metricSeed = int(i)
    seeds[counter, 0] = metricSeed
        
    counter += 1
      
    finalMetric = pd.DataFrame(results, columns = ["Correlation"])
    finalSeeds = pd.DataFrame(seeds, columns = ["Seed"])
    seedDataframe = pd.concat([finalSeeds, finalMetric], axis = 1)

In [10]:
seedDataframe.set_index('Seed')

,Correlation
Seed,
1.0,0.215303
2.0,0.240452
3.0,0.256793
4.0,0.232184
5.0,0.889542
6.0,0.217876
7.0,0.205084
8.0,0.220616
9.0,0.236988


In [11]:
# Creating empty DataFrame that will fill up after the loop
impactFactorDataframe = pd.DataFrame()
     
# Generating random seed as integer up to 10000
randomNums = np.random.randint(10000, size=(1))
randomNum = randomNums[0]
print(randomNum)

wObject = WaterMark(randomNum)

# All TIF files in the src_path are now imgs
imgsSource = srcPth.glob('*.tif')
    
# Counting the number of images in src_folder
list = os.listdir(srcFolder)
# -1 because there is one hidden file in directory
numImages = len(list) - 1

# Creating variable for range of possible impact factors
impFctRange = range(0, 10001, 5000)

# Creating zeros Numpy Array and setting counter for results
resultsMarked = np.zeros([numImages, len(impFctRange)])
resultsGCR = np.zeros([numImages, len(impFctRange)])
resultsIF = np.zeros([1, len(impFctRange)])

counter2 = 0
    
for img in imgsSource:

    counter = 0
        
    for impFct in impFctRange:
        
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Mark image with different Impact Factor values
        imgMarked = wObject.embedMark(imgOriginal, factor = impFct)
        
        # GCR Masking
        imgGCR = WaterMark.gcrMasking(imgOriginal, imgMarked)
            
        metricMarked = wObject.decodeMark(imgMarked, 'CORR')
        resultsMarked[counter2, counter] = metricMarked
        
        metricGCR = wObject.decodeMark(imgGCR, 'CORR')
        resultsGCR[counter2, counter] = metricGCR
        
        metricIF = impFct
        resultsIF[0, counter] = metricIF
        
        counter += 1
        print(counter)
        print(counter2)
            
        finalMarked = pd.DataFrame(resultsMarked)
        finalGCR = pd.DataFrame(resultsGCR)
        finalIF = pd.DataFrame(resultsIF)
        impactFactorDataframe = pd.concat([finalMarked, finalGCR], axis = 1)
    counter2 += 1

5745
1
0
2
0
3
0
1
1
2
1
3
1
1
2
2
2
3
2
1
3
2
3
3
3
1
4
2
4
3
4


In [12]:
finalMarked

,0,1,2
0,0.205526,0.719335,0.723303
1,0.237313,0.800005,0.803943
2,0.189766,0.912439,0.913797
3,0.245420,0.789698,0.789092
4,0.259863,0.951993,0.952379


In [13]:
finalGCR

,0,1,2
0,0.205196,0.538105,0.514105
1,0.237592,0.755874,0.755191
2,0.187237,0.849214,0.842315
3,0.242868,0.746993,0.747635
4,0.257541,0.919303,0.911993


In [14]:
impactFactorDataframe1 = pd.concat([finalIF, finalMarked], axis = 0)
headers = impactFactorDataframe1.iloc[0]
impactFactorDataframeMarked = pd.DataFrame(impactFactorDataframe1.values[1:], columns=headers)

impactFactorDataframe2 = pd.concat([finalIF, finalGCR], axis = 0)
headers = impactFactorDataframe2.iloc[0]
impactFactorDataframeGCR = pd.DataFrame(impactFactorDataframe1.values[1:], columns=headers)

In [16]:
impactFactorDataframeMarked

,0.0,5000.0,10000.0
0,0.205526,0.719335,0.723303
1,0.237313,0.800005,0.803943
2,0.189766,0.912439,0.913797
3,0.245420,0.789698,0.789092
4,0.259863,0.951993,0.952379


In [17]:
impactFactorDataframeGCR

,0.0,5000.0,10000.0
0,0.205526,0.719335,0.723303
1,0.237313,0.800005,0.803943
2,0.189766,0.912439,0.913797
3,0.245420,0.789698,0.789092
4,0.259863,0.951993,0.952379


In [ ]:
impactFactorFinalMetric.loc[[0, 1, 2, 3, 4], :]

In [ ]:
impactFactorFinalMetric.loc[[5, 6, 7, 8, 9], :]

In [ ]:
impactFactorFinalMetric.loc[[10, 11, 12, 13, 14], :]

In [18]:
# Creating empty DataFrame that will fill up after the loop
metricDataframe = pd.DataFrame()
for i in range(1, 2):
    
    # Generating random seed as integer up to 10000
    randomNums = np.random.randint(10000, size=(1))
    randomNum = randomNums[0]
    print(randomNum)

    wObject = WaterMark(randomNum)

    # All TIF files in the src_path are now imgs
    imgsSource = srcPth.glob('*.tif')
    
    # Counting the number of images in src_folder
    list = os.listdir(srcFolder)
    # -1 because there os one hidden file in directory
    numImages = len(list) - 1
    
    # Creating zeros Numpy Array and setting counter for results
    results = np.zeros([numImages, 10])
    counter = 0
        
    for img in imgsSource:
    
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Find impact factor within a PSNR range
        ImpactFactor = wObject.findImpactFactor(imgOriginal, rangePSNR = (35,40))
        
        # Mark image
        imgMarked = wObject.embedMark(imgOriginal, factor = ImpactFactor)
        
        #CROPPING
        #CROPPING
        #CROPPING
        
        # Width
        lx = imgMarked.shape[0]
        # Height
        ly = imgMarked.shape[1]
        
        # Generating cropped image
        # 1/8 from each side (result is by 1/4 smaller than original)
        imgCropped = imgMarked[lx // 8: - lx // 8, ly // 8: - ly // 8]
        
        #SCALING
        #SCALING
        #SCALING
        
        # Wanted value of scaled image (in pixels)
        height = 768
        width = 768
        
        # Generating scaled image
        imgScaled = cv2.resize(imgMarked, (width, height))

        #ROTATION
        #ROTATION
        #ROTATION
        
        # Rotation angle
        angle = 90
        
        # Defines center of image
        rows, cols = imgMarked.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        
        # Generating rotated image
        imgRotated = cv2.warpAffine(imgMarked, M, (cols, rows))    

        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        
        # Defining points in original image
        point1a = [50, 50]
        point1b = [200, 50]
        point1c = [50, 200]
        # Defining points in transformed image
        point2a = [10, 100]
        point2b = [200, 50]
        point2c = [100, 250]
        
        rows, cols, ch = imgMarked.shape
        # Compiling original points into single array
        pts1 = np.float32([point1a, point1b, point1c])
        # Compiling transformed points into single array
        pts2 = np.float32([point2a, point2b, point2c])
        M = cv2.getAffineTransform(pts1, pts2)
        
        #Generating affined image
        imgAffined = cv2.warpAffine(imgMarked, M, (cols, rows))
          
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        
        # Horizontal deviation (intensity)
        kernelX = 3
        # Vertical deviation (intensity)
        kernelY = 3
        
        kernelMultiplication = kernelX * kernelY
        kernel = np.ones((kernelX, kernelY), np.float32)/kernelMultiplication
        
        #Generating convoluted image
        imgConvoluted = cv2.filter2D(imgMarked, -1, kernel)  
        
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        
        # Horizontal deviation (blur intensity)
        kernelX = 3
        # Vertical deviation (blur intensity)
        kernelY = 3
        
        # Generating blurred image
        imgBlurred = cv2.GaussianBlur(imgMarked, (kernelX, kernelY), cv2.BORDER_DEFAULT)
        
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        
        # Wanted value of rotation angle (in degrees)
        noiseAmount = 0.5
        
        # Generate Gaussian noise
        gauss = np.random.normal(0, noiseAmount, imgMarked.size)
        gauss = gauss.reshape(
            imgMarked.shape[0], imgMarked.shape[1], imgMarked.shape[2]).astype('uint8')
        
        # Generating Gaussian noised image
        imgNoised = cv2.add(imgMarked, gauss)
        
        metricOriginal = wObject.decodeMark(imgOriginal, 'CORR')
        results[counter, 0] = metricOriginal
        
        metricCropped = wObject.decodeMark(imgCropped, 'CORR')
        results[counter, 1] = metricCropped
        
        metricScaled = wObject.decodeMark(imgScaled, 'CORR')
        results[counter, 2] = metricScaled
        
        metricRotated = wObject.decodeMark(imgRotated, 'CORR')
        results[counter, 3] = metricRotated
        
        metricAffined = wObject.decodeMark(imgAffined, 'CORR')
        results[counter, 4] = metricAffined
        
        metricConvoluted = wObject.decodeMark(imgConvoluted, 'CORR')
        results[counter, 5] = metricConvoluted
        
        metricBlurred = wObject.decodeMark(imgBlurred, 'CORR')
        results[counter, 6] = metricBlurred
        
        metricNoised = wObject.decodeMark(imgNoised, 'CORR')
        results[counter, 7] = metricNoised
        
        metricMarked = wObject.decodeMark(imgMarked, 'CORR')
        results[counter, 8] = metricMarked
        
        metricImpactFactor = ImpactFactor
        results[counter, 9] = metricImpactFactor
        
        counter += 1
        
        metricValues = ["Original", "Cropped", "Scaled", "Rotated", "Affined", "2D Convoluted", "Blurred", "Noised", "GCR / Marked", "Impact Factor"]
        metricSeeds = [f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}", f"Marked {randomNum}"]
        
        finalMetric = pd.DataFrame(results, columns = [metricSeeds, metricValues])
        
    metricDataframe = pd.concat([metricDataframe, finalMetric], axis = 1)

3551


In [19]:
# Creating empty DataFrame that will fill up after the loop
gcrMetricDataframe = pd.DataFrame()

for i in range(1, 2):
    
    # Using seed from previous cell
    wObject = WaterMark(randomNum)

    # All TIF files in the src_path are now imgs
    imgsSource = srcPth.glob('*.tif')
    
    # Counting the number of images in src_folder
    list = os.listdir(srcFolder)
    # -1 because there os one hidden file in directory
    numImages = len(list) - 1
    
    # Creating zeros Numpy Array and setting counter for results
    gcrResults = np.zeros([numImages, 10])
    counter = 0
        
    for img in imgsSource:
    
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Find impact factor within a PSNR range
        ImpactFactor = wObject.findImpactFactor(imgOriginal, rangePSNR = (35,40))
        
        # Mark image
        imgMarked = wObject.embedMark(imgOriginal, factor = ImpactFactor)
        
        # GCR Masking
        imgGCR = WaterMark.gcrMasking(imgOriginal, imgMarked)
        
        #CROPPING
        #CROPPING
        #CROPPING
        
        # Width
        lx = imgGCR.shape[0]
        # Height
        ly = imgGCR.shape[1]
        
        # Generating cropped image
        # 1/8 from each side (result is by 1/4 smaller than original)
        imgCropped = imgGCR[lx // 8: - lx // 8, ly // 8: - ly // 8]
        
        #SCALING
        #SCALING
        #SCALING
        
        # Wanted value of scaled image (in pixels)
        height = 768
        width = 768
        
        # Generating scaled image
        imgScaled = cv2.resize(imgGCR, (width, height))

        #ROTATION
        #ROTATION
        #ROTATION
        
        # Rotation angle
        angle = 90
        
        # Defines center of image
        rows, cols = imgGCR.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        
        # Generating rotated image
        imgRotated = cv2.warpAffine(imgGCR, M, (cols, rows))    

        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        
        # Defining points in original image
        point1a = [50, 50]
        point1b = [200, 50]
        point1c = [50, 200]
        # Defining points in transformed image
        point2a = [10, 100]
        point2b = [200, 50]
        point2c = [100, 250]
        
        rows, cols, ch = imgGCR.shape
        # Compiling original points into single array
        pts1 = np.float32([point1a, point1b, point1c])
        # Compiling transformed points into single array
        pts2 = np.float32([point2a, point2b, point2c])
        M = cv2.getAffineTransform(pts1, pts2)
        
        #Generating affined image
        imgAffined = cv2.warpAffine(imgGCR, M, (cols, rows))
    
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        
        # Horizontal deviation (intensity)
        kernelX = 3
        # Vertical deviation (intensity)
        kernelY = 3
        
        kernelMultiplication = kernelX * kernelY
        kernel = np.ones((kernelX, kernelY), np.float32)/kernelMultiplication
        
        #Generating convoluted image
        imgConvoluted = cv2.filter2D(imgGCR, -1, kernel)
       
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        
        # Horizontal deviation (blur intensity)
        kernelX = 3
        # Vertical deviation (blur intensity)
        kernelY = 3
        
        # Generating blurred image
        imgBlurred = cv2.GaussianBlur(imgGCR, (kernelX, kernelY), cv2.BORDER_DEFAULT)
  
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        
        # Wanted value of rotation angle (in degrees)
        noiseAmount = 0.5
        
        # Generate Gaussian noise
        gauss = np.random.normal(0, noiseAmount, imgGCR.size)
        gauss = gauss.reshape(
            imgGCR.shape[0], imgGCR.shape[1], imgGCR.shape[2]).astype('uint8')
        
        # Generating Gaussian noised image
        imgNoised = cv2.add(imgGCR, gauss)
      
        metricOriginal = wObject.decodeMark(imgOriginal, 'CORR')
        gcrResults[counter, 0] = metricOriginal
        
        metricCropped = wObject.decodeMark(imgCropped, 'CORR')
        gcrResults[counter, 1] = metricCropped
        
        metricScaled = wObject.decodeMark(imgScaled, 'CORR')
        gcrResults[counter, 2] = metricScaled
        
        metricRotated = wObject.decodeMark(imgRotated, 'CORR')
        gcrResults[counter, 3] = metricRotated
        
        metricAffined = wObject.decodeMark(imgAffined, 'CORR')
        gcrResults[counter, 4] = metricAffined
        
        metricConvoluted = wObject.decodeMark(imgConvoluted, 'CORR')
        gcrResults[counter, 5] = metricConvoluted
        
        metricBlurred = wObject.decodeMark(imgBlurred, 'CORR')
        gcrResults[counter, 6] = metricBlurred
        
        metricNoised = wObject.decodeMark(imgNoised, 'CORR')
        gcrResults[counter, 7] = metricNoised
        
        metricGCR = wObject.decodeMark(imgGCR, 'CORR')
        gcrResults[counter, 8] = metricGCR
        
        metricImpactFactor = ImpactFactor
        gcrResults[counter, 9] = metricImpactFactor
        
        counter += 1
        
        gcrMetricValues = ["Original", "Cropped", "Scaled", "Rotated", "Affined", "2D Convoluted", "Blurred", "Noised", "GCR / Marked", "Impact Factor"]
        gcrMetricSeeds = [f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}", f"GCR {randomNum}"]
           
        gcrFinalMetric = pd.DataFrame(gcrResults, columns = [gcrMetricSeeds, gcrMetricValues])
        
    gcrMetricDataframe = pd.concat([gcrMetricDataframe, gcrFinalMetric], axis = 1)

In [20]:
# Show Pandas DataFrame
metricDataframe

Marked 3551                                                                  \
     Original   Cropped    Scaled   Rotated   Affined 2D Convoluted   Blurred   
0    0.238385  0.412058  0.188064  0.569136  0.240259      0.658604  0.659094   
1    0.256926  0.469100  0.220732  0.633990  0.222214      0.766051  0.766438   
2    0.219449  0.650807  0.186601  0.742299  0.247187      0.867958  0.869461   
3    0.224793  0.611478  0.228483  0.669937  0.278689      0.767094  0.767442   
4    0.222426  0.685059  0.192888  0.768065  0.258274      0.925849  0.926863   

                                        
     Noised GCR / Marked Impact Factor  
0  0.663179     0.696211    1303.21000  
1  0.744367     0.771760     952.67909  
2  0.866438     0.906590    1303.21000  
3  0.742452     0.783620    1303.21000  
4  0.937020     0.945238    1114.24455

In [21]:
# Show Pandas DataFrame
gcrMetricDataframe

GCR 3551                                                                  \
   Original   Cropped    Scaled   Rotated   Affined 2D Convoluted   Blurred   
0  0.238385  0.247057  0.181616  0.383075  0.236710      0.251996  0.256367   
1  0.256926  0.348922  0.228156  0.543152  0.222561      0.656229  0.657604   
2  0.219449  0.374972  0.177002  0.579303  0.211177      0.464779  0.470840   
3  0.224793  0.574740  0.225911  0.629489  0.274023      0.681738  0.682801   
4  0.222426  0.529860  0.182777  0.716737  0.267076      0.826555  0.828445   

                                        
     Noised GCR / Marked Impact Factor  
0  0.267204     0.447657    1303.21000  
1  0.604364     0.665021     952.67909  
2  0.504184     0.710926    1303.21000  
3  0.630130     0.723813    1303.21000  
4  0.839692     0.879924    1114.24455

In [ ]:
gcrMetricDataframe.boxplot(rot=90)

In [22]:
describeDataframe = metricDataframe.describe()

In [23]:
gcrdescribeDataframe = gcrMetricDataframe.describe()

In [24]:
compareDataframe = describeDataframe.join(gcrdescribeDataframe)

In [25]:
comparemeanDataframe = compareDataframe.iloc[1]

In [26]:
comparemeanDataframeUnstacked = comparemeanDataframe.unstack(level = 0)

In [27]:
comparemeanDataframeUnstacked

,GCR 3551,Marked 3551
2D Convoluted,0.576259,0.797111
Affined,0.242309,0.249325
Blurred,0.579212,0.797860
Cropped,0.415110,0.565700
GCR / Marked,0.685468,0.820684
Impact Factor,1195.310728,1195.310728
Noised,0.569115,0.790691
Original,0.232396,0.232396
Rotated,0.570351,0.676685
Scaled,0.199092,0.203354


In [ ]:
# Colors: https://github.com/matplotlib/matplotlib/blob/38be7aeaaac3691560aeadafe46722dda427ef47/lib/matplotlib/mpl-data/stylelib/fivethirtyeight.mplstyle

plt.style.use('fivethirtyeight')

# Creating graph
fig, ax1 = plt.subplots(figsize=(12,8))

# Setting X-axis and first Y-axis
ax1.set_xlabel('Impact Factor')
ax1.set_ylabel('PSNR', color = '#008fd5')

# Creating a second Y-axis that shares the same X-axis
ax2 = ax1.twinx()  
ax2.set_ylabel('SSIM', color = '#fc4f30')

# Show grid
ax1.grid(True)
ax2.grid(True)

# Setting different colors for each variable
ax1.plot(comparemeanDataframeUnstacked['2982'], color = '#008fd5')
ax2.plot(comparemeanDataframeUnstacked['9721'], color = '#fc4f30')

In [ ]:
stackedDataframe.plot.hist(legend = True)